In [2]:
import sys
sys.path.append("../")
from mir.distances import graph
import pandas as pd

In [3]:
vdjdb = pd.read_csv('assets/vdjdb.slim.txt', sep='\t')
vdjdb[vdjdb['gene'] == 'TRB']['antigen.epitope'].value_counts()

antigen.epitope
KLGGALQAK        12667
NLVPMVATV         4697
GILGFVFTL         3501
AVFDRKSDAK        1648
ELAGIGILTV        1422
                 ...  
VMPLSAPTL            1
FGDDTVIEV            1
KQWLVWLFL            1
QHIRCNIPKRISA        1
IPTNFTISV            1
Name: count, Length: 1168, dtype: int64

In [17]:
def get_seqs(epi, size = 1000, gene = 'TRB'):
    return vdjdb[vdjdb['gene'].eq(gene) & vdjdb['antigen.epitope'].isin(epi)] \
        .groupby('antigen.epitope') \
        .apply(lambda x: x.sample(size, random_state = 42))['cdr3'] \
        .unique()

epi_seqs = get_seqs(['GLCTLVAML', 'GILGFVFTL', 'NLVPMVATV'])
print(len(epi_seqs))

2989


In [25]:
import time

start = time.time()
dist = graph.HammingDistances(epi_seqs, nproc=1)
print([x for x in dist.get_edges()][1:10])
end = time.time()
print('No parallel hamming', end - start)

start = time.time()
dist = graph.HammingDistances(epi_seqs, nproc=8, chunk_sz=4096)
print([x for x in dist.get_edges()][1:10])
end = time.time()
print('Parallel 8 process hamming', end - start)


[('CASSTRSGETQYF', 'CASSGRSGETQYF'), ('CASSTRSGETQYF', 'CASSTRAGETQYF'), ('CASSTRSGETQYF', 'CASSTRSAETQYF'), ('CANSIRSTETQYF', 'CAGSIRSTETQYF'), ('CASSIRSAYEQFF', 'CASSIRSAHEQFF'), ('CASSHRSTGELFF', 'CASSHRATGELFF'), ('CASSHRSTGELFF', 'CASSARSTGELFF'), ('CASSPRSGETQYF', 'CASSGRSGETQYF'), ('CASSSRSGVEQFF', 'CASSSRSGNEQFF')]
No parallel hamming 26.623914003372192
[('CASSTRSGETQYF', 'CASSGRSGETQYF'), ('CASSTRSGETQYF', 'CASSTRAGETQYF'), ('CASSTRSGETQYF', 'CASSTRSAETQYF'), ('CANSIRSTETQYF', 'CAGSIRSTETQYF'), ('CASSIRSAYEQFF', 'CASSIRSAHEQFF'), ('CASSHRSTGELFF', 'CASSHRATGELFF'), ('CASSHRSTGELFF', 'CASSARSTGELFF'), ('CASSPRSGETQYF', 'CASSGRSGETQYF'), ('CASSSRSGVEQFF', 'CASSSRSGNEQFF')]
Parallel 4 process hamming 5.401999235153198


In [26]:
print([x for x in dist.get_edges()][1:11])

[('CASSTRSGETQYF', 'CASSGRSGETQYF'), ('CASSTRSGETQYF', 'CASSTRAGETQYF'), ('CASSTRSGETQYF', 'CASSTRSAETQYF'), ('CANSIRSTETQYF', 'CAGSIRSTETQYF'), ('CASSIRSAYEQFF', 'CASSIRSAHEQFF'), ('CASSHRSTGELFF', 'CASSHRATGELFF'), ('CASSHRSTGELFF', 'CASSARSTGELFF'), ('CASSPRSGETQYF', 'CASSGRSGETQYF'), ('CASSSRSGVEQFF', 'CASSSRSGNEQFF'), ('CASSSRSGVEQFF', 'CASSSRSGHEQFF')]
